In [56]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gzip
import pickle

In [57]:
df1 = pd.read_csv("../00.data/scraped/meneame_scraped_1.csv", encoding="utf-8")
df2 = pd.read_csv("../00.data/scraped/meneame_scraped_2.csv", encoding="utf-8")
df3 = pd.read_csv("../00.data/scraped/meneame_scraped_3.csv", encoding="utf-8")

In [58]:
df = pd.concat([df1,df2,df3], ignore_index=True)
df

,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,scraped_date
0,4032594,RTVE mantiene que Mazón llegó al Cecopi antes ...,Los servicios informativos de la radio y la te...,https://meneame.net/story/rtve-mantiene-mazon-...,115,651,410,57,58,1,actualidad,30,2025-02-27 12:50:02,CosasQueLees,eldiario.es,https://www.eldiario.es/comunitat-valenciana/r...,2025-02-27 13:30:08
1,4032568,Arde Mississippi - ¿De donde sale todo ese odio?,Gene Hackman en la película Arde Mississippi (...,https://meneame.net/story/arde-mississippi-don...,77,982,362,50,27,2,ocio,24,2025-02-27 12:35:02,Grahml,youtube.com,https://www.youtube.com/watch?v=dnWDqcdbANQ,2025-02-27 13:30:08
2,4032569,La Guardia Civil desaloja todos los colegios d...,La Guardia Civil está desalojando preventivame...,https://meneame.net/story/guardia-civil-desalo...,97,761,369,50,47,0,actualidad,70,2025-02-27 12:10:05,Tunguska08Chelyabinsk13,levante-emv.com,https://www.levante-emv.com/sucesos/2025/02/27...,2025-02-27 13:30:08
3,4032482,La advertencia de un exdiputado socialista a M...,El exdiputado autonómico David Calvo ha lanzad...,https://meneame.net/story/advertencia-exdiputa...,141,505,352,77,64,1,actualidad,53,2025-02-27 11:30:03,jonolulu,abc.es,https://www.abc.es/espana/comunidad-valenciana...,2025-02-27 13:30:08
4,4032509,El juez Hurtado desprecia de nuevo el testimon...,"En su argumentación, el juez Hurtado “hace pro...",https://meneame.net/story/juez-hurtado-desprec...,213,335,349,99,114,1,politica,28,2025-02-27 11:05:02,candonga1,elpais.com,https://elpais.com/espana/2025-02-27/el-juez-h...,2025-02-27 13:30:08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
287500,6,Instalar Linux en iPod Mini | GPL Tarragona,Enseña a instalar Linux en el iPod Mini.\n\nMu...,https://meneame.net/story/instalar-linux-en-ip...,29,0,31,29,0,0,mnm,0,2005-12-07 12:00:01,asappla,gpltarragona.org,http://www.gpltarragona.org/node/view/395,2025-02-27 13:27:57
287501,4,Entrevista de El Mundo a Jose Antonio Marina,2. ¿Es la ética la filosofía del futuro?\nSi. ...,https://meneame.net/story/entrevista-de-el-mun...,18,0,22,18,0,0,mnm,0,2005-12-07 11:30:01,gabrielperezs,el-mundo.es,http://www.el-mundo.es/encuentros/invitados/20...,2025-02-27 13:27:57
287502,3,Prova xfce 4.2.3!,Ara que els de repositoris de totes les distri...,https://meneame.net/story/prova-xfce-423,34,0,13,34,0,0,mnm,0,2005-12-07 10:30:01,paurullan,xfce.org,http://www.xfce.org/release_notes/4.2.3.1_chan...,2025-02-27 13:27:57
287503,2,Los 84 errores de noxtrum,"Pues sí, noxtrum, el buscador de TPI y Telefón...",https://meneame.net/story/los-84-errores-de-no...,61,0,11,61,0,0,mnm,5,2005-12-07 09:25:02,FrIkI,pacoros.net,http://www.pacoros.net/diario/2005/12/07/los-8...,2025-02-27 13:27:57


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287505 entries, 0 to 287504
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   news_id          287505 non-null  int64 
 1   title            287505 non-null  object
 2   content          287488 non-null  object
 3   full_story_link  287505 non-null  object
 4   meneos           287505 non-null  int64 
 5   clicks           287505 non-null  int64 
 6   karma            287505 non-null  int64 
 7   positive_votes   287505 non-null  int64 
 8   anonymous_votes  287505 non-null  int64 
 9   negative_votes   287505 non-null  int64 
 10  category         287505 non-null  object
 11  comments         287505 non-null  int64 
 12  published_date   287505 non-null  object
 13  user             287505 non-null  object
 14  source           287505 non-null  object
 15  source_link      287505 non-null  object
 16  scraped_date     287505 non-null  object
dtypes: int64(8

In [60]:
df["published_date"] = pd.to_datetime(df["published_date"])
df["scraped_date"] = pd.to_datetime(df["scraped_date"])

In [61]:
df = df.drop_duplicates(subset=["news_id"])

# Procesamiento de 'comunidad'

In [62]:
# Diccionario de provincias-ccaa
PROVINCIAS_COMUNIDADES = {
    "Andalucía": {
        "provincias": ["Almería", "Cádiz", "Córdoba", "Granada", "Huelva", "Jaén", "Málaga", "Sevilla"]
    },
    "Aragón": {
        "provincias": ["Huesca", "Teruel", "Zaragoza"]
    },
    "Asturias": {
        "provincias": ["Asturias"]
    },
    "Islas Baleares": {
        "provincias": ["Illes Balears"],
        "equivalencias": {
            "Mallorca": "Illes Balears",
            "Menorca": "Illes Balears",
            "Cabrera": "Illes Balears",
            "Ibiza": "Illes Balears",
            "Formentera": "Illes Balears"
        }
    },
    "Canarias": {
        "provincias": ["Las Palmas", "Santa Cruz de Tenerife"],
        "equivalencias": {
            "Gran Canaria": "Las Palmas",
            "Lanzarote": "Las Palmas",
            "Fuerteventura": "Las Palmas",
            "La Graciosa": "Las Palmas",
            "Tenerife": "Santa Cruz de Tenerife",
            "La Gomera": "Santa Cruz de Tenerife",
            "El Hierro": "Santa Cruz de Tenerife",
            "La Palma": "Santa Cruz de Tenerife"
        }
    },
    "Cantabria": {
        "provincias": ["Cantabria"]
    },
    "Castilla La Mancha": {
        "provincias": ["Albacete", "Ciudad Real", "Cuenca", "Guadalajara", "Toledo"]
    },
    "Castilla y León": {
        "provincias": ["Ávila", "Burgos", "León", "Palencia", "Salamanca", "Segovia", "Soria", "Valladolid", "Zamora"]
    },
    "Cataluña": {
        "provincias": ["Barcelona", "Girona", "Lleida", "Tarragona"],
        "equivalencias": {
            "Gerona": "Girona",
            "Lérida": "Lleida"
        }
    },
    "Extremadura": {
        "provincias": ["Badajoz", "Cáceres"]
    },
    "Galicia": {
        "provincias": ["A Coruña", "Lugo", "Ourense", "Pontevedra"],
        "equivalencias": {
            "Coruña": "A Coruña"
        }
    },
    "Madrid": {
        "provincias": ["Madrid"]
    },
    "Murcia": {
        "provincias": ["Murcia"]
    },
    "Navarra": {
        "provincias": ["Navarra"]
    },
    "La Rioja": {
        "provincias": ["La Rioja"]
    },
    "País Vasco": {
        "provincias": ["Araba/Álava", "Bizkaia/Vizcaya", "Gipuzkoa/Guipúzcoa"],
        "equivalencias": {
            "Araba": "Araba/Álava",
            "Álava": "Araba/Álava",
            "Bizkaia": "Bizkaia/Vizcaya",
            "Vizcaya": "Bizkaia/Vizcaya",
            "Gipuzkoa": "Gipuzkoa/Guipúzcoa",
            "Guipúzcoa": "Gipuzkoa/Guipúzcoa"
        }
    },
    "Comunidad Valenciana": {
        "provincias": ["Alacant/Alicante", "Castelló/Castellón", "València/Valencia"],
        "equivalencias": {
            "Alicante": "Alacant/Alicante",
            "Alacant": "Alacant/Alicante",
            "Castellón": "Castelló/Castellón",
            "Castelló": "Castelló/Castellón",
            "Valencia": "València/Valencia",
            "València": "València/Valencia"
        }
    },
    "Ceuta": {
        "provincias": ["Ceuta"]
    },
    "Melilla": {
        "provincias": ["Melilla"]
    }
}

In [63]:
def asignar_provincia_comunidad(row):
    title = str(row["title"]) if pd.notna(row["title"]) else ""
    content = str(row["content"]) if pd.notna(row["content"]) else ""
    
    for comunidad, datos in PROVINCIAS_COMUNIDADES.items():
        provincias = datos["provincias"]
        equivalencias = datos.get("equivalencias", {})
        
        for provincia in provincias:
            if provincia in title or provincia in content:
                return {"provincia": provincia, "comunidad": comunidad}
        
        for variante, provincia_estandar in equivalencias.items():
            if variante in title or variante in content:
                return {"provincia": provincia_estandar, "comunidad": comunidad}

    return {"provincia": "Desconocido", "comunidad": "Desconocido"}

In [64]:
# Aplicar la función fila por fila
df.loc[:, ["provincia", "comunidad"]] = df.apply(asignar_provincia_comunidad, axis=1).apply(pd.Series)

In [65]:
df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)

df['provincia'] = df['provincia'].replace('Desconocido', np.nan)



/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_68468/470152133.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['comunidad'] = df['comunidad'].replace('Desconocido', np.nan)
/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_68468/470152133.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['provincia'] = df['provincia'].replace('Desconocido', np.nan)


# Procesamiento de Categorias


In [66]:
news_category_map = {
    "Política y Sociedad": ["EEUU", "MADRID", "Rusia", "Japón", "México", "Venezuela", "Cataluña", "Galiza", "Andalucía", "Balears", "Canarias", "CastillaLeón", "deutschland", "Barcelona", "Galicia", "Cantabria", "Aragón", "Málaga", "Roma", "INTERNAC", "Africa", "OrienteMedio", "Cuba", "politica", "corruPPción", "corruptcion", "corrupcion", "EleccionesE", "Podemos", "República", "RentaBásica", "Cayetanos", "LuchaDClases", "Anarquismo", "DDHH", "Autonomos", "Justicia7291", "Tribunales", "actualidad", "ZasSmeame", "Campañas", "Fachéame", "VIOLENCIA_G", "prensa", "borbones", "KKKantiPDMS", "AEDE", "procescatalà", "RebeliónMnm", "DchoLaboral", "Bulos", "Desinfórmame", "Madriléame", "Begoñéame", "Palestina", "Israel", "Siria", "GuerraCivilñ", "ayudaucrania", "Rescates"],
    "Tecnología y Ciencia": ["tecnología", "Informática", "GNU_LINUX", "linux", "startup", "Criptomoneda", "bitcoin", "SectorTIC", "Meteorología", "Física", "genetica", "Neurociencia", "Debian", "retrocomp", "hacking", "Metamaterial", "Micromundo", "atomo", "Impresión3D", "Drones", "netsec", "RaspberryPi", "SysDevs", "ciencia", "astronomia", "ASTRONÁUTICA", "softlibre", "Ubuntu", "android", "smartphone", "Firefox", "Blender", "IOOcia", "Marte", "Visualdata", "INVENTOS", "Numismática", "FÓSILES"],
    "Salud y Medicina": ["salud", "medicina", "Nutrición", "sanidad", "PREVENCIÓN", "HOMEOPATÍA", "VacunaGate", "vacúname", "SANIDAD", "Alimentación", "ebola", "Coronavirus"],
    "Negocios y Economía": ["Economía", "banca", "subvenciones", "CrisEnergéti"],
    "Entretenimiento y Cultura": ["Cachopos", "cervecéame", "ocio", "Cine", "Series", "Videojuegos", "Fotografía", "Música", "cinéfilos", "FolkloreÑ", "Cómics", "Webcomics", "ArteyMercado", "Arte", "Ilustracion", "clásica", "PALABROS", "Covers", "cultura", "Eurovision", "Poesia", "Bushido", "Arquitectura", "rol", "Podcasteros", "Viajar", "CiFi", "Gastronomia", "Clicómics", "Cocíname", "curiosidades", "acapela", "twitter", "reddit", "museo", "Relatocorto", "Autorrelatos", "Reportajes", "Entrevistas", "microrelatos", "Hazlotumismo", "Los12monos", "Pregúntame", "Recomiéndame", "LETREROS", "Escúchame", "Documentales", "STARWARS", "Ochenteando", "UHF", "Preguntas", "Fotomundo", "roleros", "metal", "B.S.O.", "TEMAZOS", "Comiñas", "MundoCelta", "Artículos", "sabíasque", "paléame"],
    "Deportes": ["deportes", "Natación", "Motociclismo", "ciclismo"],
    "Medioambiente y Energía": ["veganismo", "MenteAnimal", "Abandonos", "MUNDO_PROTE", "MAmbiente", "Cambioclima", "Ecología", "Energias", "sequía", "Naturaleza", "Botánica", "PLANTAS", "AgroInfo", "Vivienda", "Animales", "gatos", "Biologia", "Faros", "mascotas", "Cáñamo", "Arañas"],
    "Historia y Humanidades": ["Historia", "Mitología", "Arqueología", "Egiptologia", "Viñetas", "Hallazgos", "Asturias", "Jurídicas", "Etimologia", "Psicología", "Psicópatas", "Filosofía", "Efemérides", "Helénica", "Hemeroteca", "Fragmentos"],
    "Humor y Memes": ["Humor", "memes", "Sarcasmos", "Chorridolias", "Tendenciosos", "Ayuséame", "podríame", "Fakeame", "Fakejóo", "Idiocracia", "meneametoday", "TuMeme", "Cosasguays", "friki", "procrastrina", "NoMundoToday", "Memeneame", "Ardilléame", "Toréame", "OYOYOY", "gilipolleces", "chorradas"],
    "Transporte": ["aviacion", "aviones", "trenes", "DIRIGIBLES", "Submarino", "Motor", "Avionéame", "BARCOS", "Tractores", "movilidad", "alas"],
    "Educación": ["Educación", "idiomas", "Enseñanza", "Literatura", "Libros", "lenguas", "Matemáticas"],
    "Crimen": ["INV", "Gamonal", "Sectas", "Sucesos"],
    "Cuestiones Sociales": ["Asperger", "Crianza", "Drogolegas", "Feminismo", "LGBT", "Derecho", "SEXOLOGÍA", "mujeresenlah", "Abuso_Animal", "religion", "laicismo"]
}

In [68]:
category_lookup = {word.lower(): category for category, words in news_category_map.items() for word in words}

# Function to map category names
def map_category(category):
    return category_lookup.get(category.lower(), "Otros")

# Apply mapping
df["category"] = df["category"].apply(map_category)


/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_68468/3119110777.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["category"] = df["category"].apply(map_category)


In [69]:
df['category'].value_counts()

category
Otros                        135754
Política y Sociedad           95349
Entretenimiento y Cultura     40448
Tecnología y Ciencia          14511
Historia y Humanidades          294
Humor y Memes                   245
Transporte                      221
Medioambiente y Energía         200
Salud y Medicina                137
Crimen                           73
Cuestiones Sociales              69
Negocios y Economía              22
Educación                        19
Deportes                         13
Name: count, dtype: int64

### categorisando "otros" usando ML

In [51]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download("stopwords")
spanish_stopwords = stopwords.words("spanish")


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucija/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [53]:
df = df.dropna(subset=["content"])
df_unlabeled = df[df['category'] == "Otros"]
df_labeled = df[df['category'] != "Otros"]

In [ ]:
le = LabelEncoder()
df_labeled["category_encoded"] = le.fit_transform(df_labeled["category"])

X = df_labeled.drop(columns=["category", "category_encoded"])  # Features
y = df_labeled["category_encoded"]  # Target (encoded categories)


In [54]:
# Vectorization
vectorizer = TfidfVectorizer(stop_words=spanish_stopwords, max_features=50000)
X = vectorizer.fit_transform(df_labeled["content"])
y = df_labeled["category"]

# Label Encoding
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)  # Convert categories to numbers

# Class distribution before sampling
print("Before Sampling:", Counter(y_encoded))

Before Sampling: Counter({np.int64(9): 95349, np.int64(4): 40438, np.int64(11): 14510, np.int64(5): 294, np.int64(6): 239, np.int64(12): 221, np.int64(7): 200, np.int64(10): 137, np.int64(0): 73, np.int64(1): 69, np.int64(8): 22, np.int64(3): 19, np.int64(2): 13})


In [91]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords

# Download stopwords
nltk.download("stopwords")
spanish_stopwords = stopwords.words("spanish")

# Drop NaN values from 'content'
df = df.dropna(subset=["content"])

# Split labeled and unlabeled data
df_unlabeled = df[df['category'] == "Otros"]
df_labeled = df[df['category'] != "Otros"]

# Encode category labels
label_encoder = LabelEncoder()
df_labeled["category_encoded"] = label_encoder.fit_transform(df_labeled["category"])

# Extract features (content) and target (category_encoded)
X = df_labeled["content"]  # Only text column
y = df_labeled["category_encoded"]

# Convert text data into numerical features using TF-IDF
vectorizer = TfidfVectorizer(stop_words=spanish_stopwords, max_features=50000)
X_tfidf = vectorizer.fit_transform(X)

# Class distribution before sampling
print("Before Sampling:", Counter(y))

# Define sampling strategies
undersampling_strategy = {9: 8000, 4: 8000, 11: 4000}  # Reduce majority classes
oversampling_strategy = {5: 2000, 6: 2000, 12: 1000, 7: 2000, 10: 1000, 
                         0: 1000, 1: 1000, 8: 1000, 3: 1000, 2: 1000}  # Increase minority classes

# Apply undersampling first, then oversampling
undersampler = RandomUnderSampler(sampling_strategy=undersampling_strategy, random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X_tfidf, y)

oversampler = SMOTE(sampling_strategy=oversampling_strategy, random_state=42)
X_resampled, y_resampled = oversampler.fit_resample(X_resampled, y_resampled)

# Print final class distribution
print("After Sampling:", Counter(y_resampled))

# Convert y_resampled back to category names
df_balanced = pd.DataFrame({"category": label_encoder.inverse_transform(y_resampled)})

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Train Logistic Regression
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# Make Predictions
y_pred = model.predict(X_test)

# Evaluate Model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.4f}")
print(classification_report(y_test, y_pred))


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lucija/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_68468/701450850.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_labeled["category_encoded"] = label_encoder.fit_transform(df_labeled["category"])


Before Sampling: Counter({9: 95349, 4: 40438, 11: 14510, 5: 294, 6: 239, 12: 221, 7: 200, 10: 137, 0: 73, 1: 69, 8: 22, 3: 19, 2: 13})
After Sampling: Counter({4: 8000, 9: 8000, 11: 4000, 5: 2000, 6: 2000, 7: 2000, 0: 1000, 1: 1000, 2: 1000, 3: 1000, 8: 1000, 10: 1000, 12: 1000})
Model Accuracy: 0.8017
              precision    recall  f1-score   support

           0       1.00      0.99      0.99       200
           1       1.00      0.98      0.99       200
           2       1.00      0.99      1.00       200
           3       1.00      1.00      1.00       200
           4       0.64      0.76      0.69      1600
           5       1.00      0.95      0.98       400
           6       0.88      0.98      0.93       400
           7       1.00      0.98      0.99       400
           8       1.00      1.00      1.00       200
           9       0.73      0.77      0.75      1600
          10       0.97      0.97      0.97       200
          11       0.78      0.38      0.51    

In [92]:
# Transform df_unlabeled content using the trained vectorizer
X_unlabeled = vectorizer.transform(df_unlabeled["content"])

# Get probabilities for each class
y_proba = model.predict_proba(X_unlabeled)

# Get the highest probability per instance
max_probs = np.max(y_proba, axis=1)

# Get the predicted category index
y_pred_indices = np.argmax(y_proba, axis=1)

# Decode categories
y_pred_labels = label_encoder.inverse_transform(y_pred_indices)

# Apply probability threshold: If prob < 50%, assign "Otros"
final_categories = np.where(max_probs >= 0.50, y_pred_labels, "Otros")

# Assign to df_unlabeled
df_unlabeled["category"] = final_categories


/var/folders/x1/dfk_lr0j0pg5th31583lwbxc0000gn/T/ipykernel_68468/1144273884.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled["category"] = final_categories


In [93]:
pd.set_option("display.max_colwidth", None)
df_unlabeled.sample(n=20)


,news_id,title,content,full_story_link,meneos,clicks,karma,positive_votes,anonymous_votes,negative_votes,category,comments,published_date,user,source,source_link,scraped_date,provincia,comunidad
204347,1096487,El Frente Polisario llama a las armas en el Sahara a partir del 21 de noviembre 2010,"Tras las reuniones y protestas realizadas en Tinduf, campamento de refugiados saharauis situado en la zona fronteriza de Marruecos con Argelia, y las tibias respuestas a lo que ya se considera una masacre por parte de los Gobiernos de España y la Unión Europea, el Frente Polisario ha llamado para estar listos para la batalla a ""todos los guerreros"" y a los que quieran unirse a su escuela militar, para el día 21 de noviembre de 2010.",https://meneame.net/story/frente-polisario-llama-armas-sahara-partir-21-noviembre-2010,586,1180,486,228,358,6,Política y Sociedad,93,2010-11-17 14:55:02,--77058--,news.suite101.net,http://news.suite101.net/article.cfm/frente-polisario-llama-a-las-armas-en-sahara-16-noviembre-2010-a30345,2025-02-27 11:21:13,NaN,NaN
190217,1369503,Guía rápida de autodefensa para trabajadores de informática,"Todas las personas que trabajamos en el sector de la informática y consultoría hemos vivido unos inicios bastante parecidos: Te engañan con promesas que ves incumplidas una tras otra, te manipulan para que hagas horas, para que no conozcas tus derechos y un largo etcétera. Hemos agrupado unas primeras informaciones de urgencia que iremos haciendo llegar a los trabajadores y trabajadoras, de esta forma intentaremos atacar un modelo de negocio basado en la hiperexplotación de trabajadores, especialmente de los que tienen poca experiencia.",https://meneame.net/story/guia-rapida-autodefensa-trabajadores-informatica,626,13809,487,263,363,4,Otros,114,2011-09-12 10:05:02,poyeur,nodo50.org,http://www.nodo50.org/coord-informatica/?q=content/gu%C3%AD-r%C3%A1pida-de-autodefensa-para-trabajadores-de-inform%C3%A1tica,2025-02-27 11:00:21,NaN,NaN
239563,558606,Un queso tan apestoso que su venta es ilegal,"[c&p] ... De carácter muy fuerte y extremadamente tradicional -se debe comer en fetas muy finas acompañado de un pan muy fuerte denominado carta da musica, y un vino tinto de buen carácter y espirituoso,- siendo popularmente considerado como altamente afrodisiaco, el mismo ha sido declarado ilegal, siendo su venta fuertemente multada por las autoridades, ya que las larvas, capaces de saltar 15 centímetros cuando son molestadas, poseen una notable resistencia a los ácidos estomacales. No obstante, y por fortuna, el mismo puede aun ser conseguido",https://meneame.net/story/queso-tan-apestoso-venta-ilegal,312,0,682,157,155,1,Otros,69,2008-12-29 18:30:02,mezvan,anfrix.com,http://www.anfrix.com/2008/12/un-queso-tan-apestoso-que-su-venta-es-ilegal/,2025-02-27 12:13:37,NaN,NaN
198624,1203599,Estudiante arrestado por protestar contra las inspecciones aeroportuarias demanda a las autoridades aéreas de USA [EN],"Cuando Aaron Tobey fue registrado en el aeropuerto de Richmond, se reveló al desvestirse que había escrito en su pecho el texto de la Cuarta Enmienda de la Constitución estadounidense, que prohíbe ""búsquedas y registros irrazonables contra los ciudadanos"", y fue arrestado por esta protesta durante noventa minutos. Ahora, está demandando a las autoridades de tráfico aéreo de USA, la secretaria de seguridad Janet Napolitano, y el aeropuerto de Richmond, por violación de las enmiendas cuarta, primera, y decimocuarta en su contra.",https://meneame.net/story/estudiante-arrestado-protestar-contra-inspecciones-demanda-usa,856,10086,554,382,474,2,Política y Sociedad,42,2011-03-18 14:15:02,Nirgal,aolnews.com,http://www.aolnews.com/2011/03/15/aaron-tobey-student-with-4th-amendment-on-chest-sues-over-airp/,2025-02-27 11:12:47,NaN,NaN
236766,597856,Tele 5 decide explotar el caso del concursante parricida de Antena 3,"A raíz de la polémica suscitada esta semana por el concursante que abandonó el reality de Antena 3 ""La vuelta al mundo"" tras cono

In [95]:
df_unlabeled['category'].value_counts()

category
Política y Sociedad          49200
Otros                        45685
Entretenimiento y Cultura    31779
Tecnología y Ciencia          8158
Humor y Memes                  919
Salud y Medicina                 7
Transporte                       3
Historia y Humanidades           2
Medioambiente y Energía          1
Name: count, dtype: int64

In [96]:
df['category'].value_counts()

category
Otros                        135754
Política y Sociedad           95349
Entretenimiento y Cultura     40438
Tecnología y Ciencia          14510
Historia y Humanidades          294
Humor y Memes                   239
Transporte                      221
Medioambiente y Energía         200
Salud y Medicina                137
Crimen                           73
Cuestiones Sociales              69
Negocios y Economía              22
Educación                        19
Deportes                         13
Name: count, dtype: int64

In [15]:
df = df.astype({
    "meneos": "uint16",
    "karma": "uint16",
    "positive_votes": "uint16",
    "negative_votes": "uint16",
    "anonymous_votes": "uint16",
    "comments": "uint16",
    "clicks": "int32",
    "category": "category",
    "provincia": "category",
    "comunidad": "category"
})

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287191 entries, 0 to 287190
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   news_id          287191 non-null  int64         
 1   title            287191 non-null  object        
 2   content          287191 non-null  object        
 3   full_story_link  287191 non-null  object        
 4   meneos           287191 non-null  uint16        
 5   clicks           287191 non-null  int32         
 6   karma            287191 non-null  uint16        
 7   positive_votes   287191 non-null  uint16        
 8   anonymous_votes  287191 non-null  uint16        
 9   negative_votes   287191 non-null  uint16        
 10  category         287191 non-null  category      
 11  comments         287191 non-null  uint16        
 12  published_date   287191 non-null  datetime64[ns]
 13  user             287191 non-null  object        
 14  source           287

In [ ]:
with gzip.open("../00.data/preprocesado/meneame_procesado.pkl.gz", "wb") as f:
    pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)
